[View in Colaboratory](https://colab.research.google.com/github/sungreong/Pymc3_bayseian/blob/master/PyMC3_and_Theano.ipynb)

In [4]:
!pip install git+https://github.com/pymc-devs/pymc3

  Cloning https://github.com/pymc-devs/pymc3 to /tmp/pip-req-build-nozzy28v
    100% |████████████████████████████████| 2.8MB 9.0MB/s 
    100% |████████████████████████████████| 184kB 14.2MB/s 
    100% |████████████████████████████████| 51kB 18.8MB/s 
  Running setup.py bdist_wheel for pymc3 ... - \ | / - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-c3rkmqzf/wheels/a1/e3/71/a696f2af0dd40bccc0ce672d616d21cc9dd90dbdd3f03242f4
  Running setup.py bdist_wheel for theano ... - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/a7/8a/6c/aad33a6144dc30cc65d437c444090076136d2c0f289f3ce183
Successfully built pymc3 theano


## Reference  : https://docs.pymc.io/theano.html

---



배열 연산과 선형 대수학을 포함하는 함수를 정의할 때 **Theano**를 사용한다.   
우리가 pymc3 모델에서 정의할 때 상수 요인에 사후 확률 밀도에서 우리의 파라미터 공안으로부터 theano function을 만든다.  
우리는 그때 이 함수의 심볼 조작을 gradient 접근으로 사용할 것이다.  
theano에 대해서 구체적인 지식은 필요없다.  
그러나 간단하게 어떻게 작동하는지는 알아보자 

$ f:R×R^n×N^n→R  \\
(a,x,y)↦∑exp(ax_{i}^{3}+y_{i}^{2}). $ 

In [14]:
import theano
import theano.tensor as tt
## 인풋 변수의 타입을 구체화 할 필요는 없다. 그래서 우리는 어떤 특정 것을 지정안하면 float64를 사용 할 것이다.

a = tt.scalar("a")
x = tt.vector("x")

## tt.ivector 은 정수 심볼 벡터를 만든다

y= tt.ivector("y")



x

In [0]:
## 다음으로 우리는 우리 함수의 출력물의 심볼 표현을 설계하기 위해 변수를 사용 할 것이다. 
## 여기서 실제로 계산은 없다. 우리는  단지 출력물을 계산하기 위해 필요한 연산만 기록해 논다.

inner = a * x**3 + y**2
out = tt.exp(inner).sum()

## tt.exp 가 신기하게도  np.exp를 사용해도 가능하다. 
## Theano 변수들은 많은 수의 연산을 할 수 있다. 우리는 여전히 numpy 대신에 theano를 선호한다. 
## 그것은 더 명백한 연산 대신에 심볼 인풋으로 하는 것이 더 명쾌하다.


In [16]:

func = theano.function([a, x, y], [out])


In [0]:
import numpy as np
a_val = 1.2
np.random.randn()
x_vals = np.random.randn(10).astype(theano.config.floatX)
y_vals = np.random.randn(10).astype(theano.config.floatX)

out = func(a_val, x_vals, y_vals)

In [0]:
a = tt.vector('a')

## (a > 0).all() isn’t actually a boolean as it would be in NumPy, but still a symbolic variable. 

if (a > 0).all():
    b = tt.sqrt(a)
else:
    b = -a

In [0]:
a = tt.vector('a')
b = tt.sqrt(a)

a = tt.vector('a')
b = tt.switch((a > 0).all(), tt.sqrt(a), -a)

In [22]:
a = tt.vector('a')
# Access the 10th element. This will fail when a function build
# from this expression is executed with an array that is too short.
b = a[10]

# Extract a subvector
b = a[[1, 2, 10]]


AdvancedSubtensor1.0

In [0]:
a = tt.vector('a')
b = tt.set_subtensor(a[:10], 1)

# is roughly equivalent to this (although theano avoids
# the copy if `a` isn't used anymore)
a = np.random.randn(10)
b = a.copy()
b[:10] = 1

## How PyMC3 uses Theano

In [0]:
import pymc3 as pm
true_mu = 0.1
data = true_mu + np.random.randn(50)

with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sd=1)
    y = pm.Normal('y', mu=mu, sd=1, observed=data)

$ mu 와 y $ 를 정의하고  
1. 추론하기 원하는 자유변수
2. 관측된 데이터
3. 함수를 만들기 위해 필요한 posterior sampling 

$ logP(μ|y)+C=logP(y|μ)+logP(μ)=:logp(μ)  \\
log(p):R→R \\ 
μ↦logN(μ|0,1)+logN(y|μ,1), $

## 우리가 추적해야 할 것 2가지
1. The parameter space (the free variables) 
2. the logp function

For each free variable we generate a Theano variable   
And for each variable (observed or otherwise) we add a term to the global logp




```
# For illustration only, those functions don't actually exist
# in exactly this way!
model = pm.Model()

mu = tt.scalar('mu')
model.add_free_variable(mu)
model.add_logp_term(pm.Normal.dist(0, 1).logp(mu))

model.add_logp_term(pm.Normal.dist(mu, 1).logp(data))
```

## Continuous variables with support only on a subset of the real numbers are treated a bit differently.
###  We create a transformed variable that has support on the reals and then modify this variable. For example:

```
with pm.Model() as model:
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd', 1)
    y = pm.Normal('y', mu=mu, sd=sd, observed=data)

```
##  is roughly equivalent to this:  
```
# For illustration only, not real code!
model = pm.Model()
mu = tt.scalar('mu')
model.add_free_variable(mu)
model.add_logp_term(pm.Normal.dist(0, 1).logp(mu))

sd_log__ = tt.scalar('sd_log__')
model.add_free_variable(sd_log__)
model.add_logp_term(corrected_logp_half_normal(sd_log__))

sd = tt.exp(sd_log__)
model.add_deterministic_variable(sd)

model.add_logp_term(pm.Normal.dist(mu, sd).logp(data))
```


The return values of the variable constructors are subclasses of theano variables,   
so when we define a variable we can use any theano operation on them:


```
design_matrix = np.array([[...]])
with pm.Model() as model:
    # beta is a tt.dvector
    beta = pm.Normal('beta', 0, 1, shape=len(design_matrix))
    predict = tt.dot(design_matrix, beta)
    sd = pm.HalfCauchy('sd', beta=2.5)
    pm.Normal('y', mu=predict, sd=sd, observed=data)
```

